# Cleaning API

We start from the code below

In [1]:
def create_data(nx, ny):
    # physics
    lx = 250000
    ly = 200000  # domain size [m]
    B0 = 3500            # mean height [m]
    β = 0.01            # mass-balance slope (data)
    c = 2.0             # mass-balance limiter
    
    # numerics
    dx, dy = lx / nx, ly / ny
    
    # initial conditions (data)
    xc = np.arange(-lx / 2 + dx / 2, lx / 2 - dx / 2, dx)
    yc = np.arange(-ly / 2 + dy / 2, ly / 2 - dy / 2, dy)
    
    Xc, Yc = np.meshgrid(xc, yc)
    
    B = B0 * np.exp(-Xc**2 / 1e10 - Yc**2 / 1e9) + B0 * np.exp(-Xc**2 / 1e9 - (Yc - ly / 8)**2 / 1e10)
    ELA = 2150 + 900 * np.arctan(Yc / ly)
    
    return B, ELA, β, c, dx, dy, xc, yc


This code has several issues:
- We should use better variables names
- some constants are unecessary defined inside the method and use on the return
- It should do only one thing

We can create a class to handle only the data from the grid

In [13]:
class DataG():
    
    def __init__(self, nx,lx, ny, ly):
        self.lx = lx
        self.ly = ly
        self.dx = lx / nx
        self.dy = ly / ny
        self.xc = np.arange(-lx / 2 + dx / 2, lx / 2 - dx / 2, dx)
        self.yc = np.arange(-ly / 2 + dy / 2, ly / 2 - dy / 2, dy)
        self.Xc, self.Yc = np.meshgrid(self.xc, self.yc)

In [14]:
def bedrock_elevation(data, mean_height):
    
    bedrock_el = mean_height * np.exp(-data.Xc**2 / 1e10 - data.Yc**2 / 1e9)
    bedrock_el += mean_height * np.exp(-data.Xc**2 / 1e9 - (data.Yc - data.ly / 8)**2 / 1e10)
    return bedrock_el

In [15]:
def eq_line_altitude(data):
    return 2150 + 900 * np.arctan(Yc / ly) 

In [1]:
import numpy as np
data = DataG(100, 100, 250000, 250000)

ModuleNotFoundError: No module named 'numpy'

In [18]:
def iceflow_solver(m_balance_slope, m_balance_limiter, mean_height, data):
    # physics
    ρg = 910.0 * 9.81    # ice density x gravity
    dt = 0.1             # time step [yr]
    # numerics
    B = bedrock_elevation(data, mean_height)
    ELA = eq_line_altitude(data)
    nx, ny = B.shape      # numerical grid resolution
    nt = 1e4              # number of time steps
    nout = 1e3            # visu and error checking interval
    ϵ = 1e-4             # steady state tolerance
    # preprocess
    a1 = 1.9e-24 * pow(ρg,3) * 31557600
    a2 = 5.7e-20 * pow(ρg,3) * 31557600
    # initialise
    S = np.zeros((nx, ny))#,np.float128)
    dSdx = np.zeros((nx-1, ny))#,np.float128)
    dSdy = np.zeros((nx, ny-1))#,np.float128)
    Snorm = np.zeros((nx-1, ny-1))#,np.float128)
    D = np.zeros((nx-1, ny-1))#,np.float128)
    qx = np.zeros((nx-1, ny-2))#,np.float128)
    qy = np.zeros((nx-2, ny-1))#,np.float128)
    H = np.zeros((nx, ny))#,np.float128)
    M = np.zeros((nx, ny))#,np.float128)
    H0 = np.zeros((nx, ny))#,np.float128)
    # time loop
    for it in range(int(nt)):
        np.copyto(H0, H)
        S = B + H
        M = np.minimum(β * (S - ELA), c)
        compute_D(D, H, S, dSdx, dSdy, Snorm, a1, a2, data.dx, data.dy)
        qx[:] = avy(D) * np.diff(S[:, 1:-1], axis=0) / dx
        qy[:] = avx(D) * np.diff(S[1:-1, :], axis=1) / dy
        H[1:-1, 1:-1] = np.maximum(H[1:-1, 1:-1] + dt * (np.diff(qx, axis=0) + np.diff(qy, axis=1) + M[1:-1, 1:-1]), 0.0)
        if it % nout == 0:
            # error checking
            #print(H[2,3:10])
            err = np.max(np.abs(H - H0))
            print(f"it = {it}, err = {err:.3e}")
            if err < ϵ:
                break
    return H, S 

In [ ]:
def visualise(H, S, data):
    S_v = np.copy(S)
    S_v[H <= 0.01] = np.nan
    fig = plt.figure(figsize=(100, 45))
    axs = fig.add_subplot(121, projection='3d')
    axs.set_xlabel("x [km]")
    axs.set_ylabel("y [km]")
    axs.set_zlabel("elevation [m]")
    xic, yic = np.meshgrid(xc, yc)
    axs.set_box_aspect((4, 4, 1))
    axs.view_init(azim=25)
    p1 = axs.plot_surface(xic / 1e3, yic / 1e3, B, rstride=1, cstride=1, cmap='viridis', edgecolor='none')
    p2 = axs.plot_surface(xic / 1e3, yic / 1e3, S_v, rstride=1, cstride=1, cmap='viridis', edgecolor='none')
    norm = mpl.colors.Normalize(vmin=0, vmax=6000)
    fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap='viridis'),
             ax=axs, orientation='vertical', label='H ice [m]', shrink=0.5)

    plt.tight_layout()
    plt.show()

# Remarks

- There are methods that take a parameter and use it only on the return
- Some methods are just auxiliar methods and should be private and not expose to users
- Redudant naming: module called ice_solver and library call iceflow, it should be iceflow.solver
- For me the library should expose two methods: solver and visualise
- Unnecessary refactor of simple operation. Some methods are too simple
- Numbers appear without doc, it is better to use CONSTANTS